In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
#import hdf5storage
import csv


Autosaving every 180 seconds


In [15]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import combinations

#############################
def plot_pairwise_interactions(locs):
    
    x_ticks=['female','male','pup1','pup2']
    text_clr = 'red'

    distance_threshold = 250 # # of pixels away assume 1 pixel ~= 0.5mm -> 20cm
    time_window = 1*25 # no of seconds to consider
    smoothing_window = 3
    min_distance = 25 # number of frames window

    traces_23hrs = locs
    #print (traces_23hrs[0].shape)
    
    # COMPUTE PAIRWISE INTERACTIONS
    animals=np.arange(locs.shape[0])
    interactions = np.zeros((animals.shape[0],animals.shape[0]),'int32') + np.nan
    durations_matrix = np.zeros((animals.shape[0], animals.shape[0]),'int32') + np.nan

    # loop over all pairwise combinations
    pair_interaction_times = []
    pairs1 = list(combinations(animals,2))
    for ctr_x, pair in enumerate(pairs1):
        traces = []
        
        # smooth out traces;
        for k in pair:
            traces1=traces_23hrs[k].copy()
            traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
            traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
            traces1 = traces1
            traces.append(traces1)

        # COMPUTE PAIRWISE DISTANCES AND NEARBY TIMES POINTS
        idx_array = []
        diffs = np.sqrt((traces[0][:,0]-traces[1][:,0])**2+
                        (traces[0][:,1]-traces[1][:,1])**2)
        idx = np.where(diffs<distance_threshold)[0]

        # COMPUTE TOTAL TIME TOGETHER
        #print ("Pairwise: ", pair, idx.shape)
        durations_matrix[pair[0],pair[1]]=idx.shape[0]/25

        # COMPUTE # OF INTERACTIONS; 
        diffs_idx = idx[1:]-idx[:-1]
        idx2 = np.where(diffs_idx>5)[0]
        interactions[pair[0],pair[1]]=idx2.shape[0]
        
        # SAVE TIMES OF INTERACTION
        pair_interaction_times.append(idx)

    # SYMMETRIZE MATRICES
    for k in range(durations_matrix.shape[0]):
        for j in range(durations_matrix.shape[1]):
            if np.isnan(durations_matrix[k,j])==False:
                durations_matrix[j,k]=durations_matrix[k,j]
                interactions[j,k]=interactions[k,j]


    #############################
    # COMPUTE TRIPLE INTERACTIONS

    multi_animal_durations = [] #np.zeros((animals.shape[0]+4),'int32') + np.nan
    names_multi_animal =[]
    ctr=0

    pairs3 = list(combinations(animals,3))
    triples_interaction_time = []
    for ctr_x, pair in enumerate(pairs3):
        names_multi_animal.append(names[pair[0]]+"\n"+ names[pair[1]]+"\n"+names[pair[2]]+"\n")
        #names_multi_animal.append(names[pair[0]]+" - "+ names[pair[1]]+" - "+names[pair[2]])
        traces = []

        # COMPUTE LOCATIONS
        for k in pair:
            traces1=traces_23hrs[k].copy()
            traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
            traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
            traces1 = traces1
            traces.append(traces1)

        # COMPUTE PAIRWISE DISTANCES AND NEARBY TIMES POINTS
        idx_array = []
        pairs2 = list(combinations(np.arange(3),2))
        for pair_ in pairs2:
            #print ("pair_", pair_)
            diffs = np.sqrt((traces[pair_[0]][:,0]-traces[pair_[1]][:,0])**2+
                            (traces[pair_[0]][:,1]-traces[pair_[1]][:,1])**2)
            idx_temp = np.where(diffs<distance_threshold)[0]
            #print ("pair_: ", pair_, idx_temp.shape)
            idx_array.append(idx_temp)

        # COMPUTE OVERLAP 
        idx3 = np.unique(np.hstack(set.intersection(*[set(x) for x in idx_array])))
        #print (pair, "IDX3 4: ", len(idx3))
        multi_animal_durations.append(len(idx3)/25)
        
        # SAVE TIMES OF INTERACTION
        triples_interaction_time.append(idx3)
        
    print ("")
    
    #################################
    # COMPUTE 4 ANIMAL INTERACTIONS
    pairs = list(combinations(animals,4))
    quad_interaction_times = []
    for ctr_x, pair in enumerate(pairs):
        names_multi_animal.append(names[pair[0]]+"\n"+ names[pair[1]]+"\n"+names[pair[2]]+"\n"+names[pair[3]])
        #names_multi_animal.append(names[pair[0]]+" - "+ names[pair[1]]+" - "+names[pair[2]]+"  - "+names[pair[3]])
        #print (pair)
        traces = []
        for k in pair:
            traces1=traces_23hrs[k].copy()
            traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
            traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
            traces1 = traces1
            traces.append(traces1)

        # loop over all combinations and get distances
        pairs2 = list(combinations(animals,2))
        idx_array = []
        for pair2 in pairs2:
            diffs = np.sqrt((traces[pair2[0]][:,0]-traces[pair2[1]][:,0])**2+
                             (traces[pair2[0]][:,1]-traces[pair2[1]][:,1])**2)
            idx_temp = np.where(diffs<distance_threshold)[0]
            #print ("pair2: ", pair2, idx_temp.shape)
            idx_array.append(idx_temp)

        # COMPUTE TOTAL TIME TOGETHER
        #print ("idx_array: ", idx_array)
        idx3 = set.intersection(*[set(x) for x in idx_array])
        #print ("4 animals 4: ", len(idx3))
        multi_animal_durations.append(len(idx3)/25)

        # SAVE TIMES OF INTERACTION
        quad_interaction_times.append(list(idx3)  )
        
        ctr+=1
    #print ("")
 

    #####################################
    ######### PLOT RESULTS  ########
    #####################################
  


    # PLOT MULTI ANIMAL DURATIONS
    fig=plt.figure()
    ax3=plt.subplot(1,1,1)
    handles, labels = ax3.get_legend_handles_labels()

    import matplotlib.patches as mpatches

    for k in range(5):
        plt.bar(k, multi_animal_durations[k], 0.9, color=clrs[k])

        patch = mpatches.Patch(color=clrs[k], label=names_multi_animal[k])

        # handles is a list, so append manual patch
        handles.append(patch) 

    # plot the legend   
    plt.legend(handles=handles,fontsize=20)
    plt.xticks([])
    plt.tick_params(labelsize=20)
    plt.xlim(-0.5,4.5)
    plt.ylabel("time together (sec)",fontsize=20)

    #ax3.set_title("Duration Multi-animal interactions",fontsize=15)
    #################################################
    ######### PLOT INTERACTIONS PAIRWISE ############
    #################################################
    fig=plt.figure()
    ax1=plt.subplot(1,2,1)

    im = plt.imshow(interactions, cmap='viridis')
    #print(locs.shape[0], x_ticks)
    plt.xticks(np.arange(locs.shape[0]), x_ticks,rotation=15)
    plt.yticks(np.arange(locs.shape[0]), x_ticks,rotation=75)
    plt.tick_params(labelsize=20)
    ax1.set_title("# of interactions",fontsize=20)
    plt.tick_params(labelsize=20)

    cbar = plt.colorbar()
    cbar.set_label("# interactions", fontsize=20)

    # Loop over data dimensions and create text annotations.
    if False:
        for i in range(interactions.shape[0]):
            for j in range(interactions.shape[1]):
                if np.isnan(durations_matrix[i, j])==True:
                    continue
                text = ax1.text(j, i, interactions[i, j],
                               ha="center", va="center", color=text_clr,
                               fontsize=13)        

    ##############################################
    ############ PLOT PAIRWISE DURATIONS ########
    #################################################
    ax2=plt.subplot(1,2,2)
    im = plt.imshow(durations_matrix, cmap='viridis')

    x_ticks=['female','male','pup1','pup2']
    plt.xticks(np.arange(locs.shape[0]), x_ticks,rotation=15)
    plt.yticks(np.arange(locs.shape[0]), x_ticks,rotation=75)
    plt.tick_params(labelsize=20)

    cbar = plt.colorbar()
    cbar.set_label("time together (sec)", fontsize=20)


    # Loop over data dimensions and create text annotations.
    if False:
        for i in range(durations_matrix.shape[0]):
            for j in range(durations_matrix.shape[1]):
                if np.isnan(durations_matrix[i, j])==True:
                    continue
                text = ax2.text(j, i, durations_matrix[i, j],
                               ha="center", va="center", color=text_clr,
                               fontsize=13)

    ax2.set_title('time together (sec)',fontsize=20)

    plt.suptitle("Behav analysis for Cohort #1\n March 16: 2020-3-16_12_54_07_193951_compressed (1hr video; "+
                 str(np.round(smoothing_window/25,2))+"sec smoothing window)",fontsize=15)

    plt.show()   

    return pair_interaction_times,triples_interaction_time, quad_interaction_times
    

def ethogram_social_huddling(pair_interaction_times,
                            triples_interaction_time,
                            quad_interaction_times):
    
    import matplotlib.patches as mpatches    
    
    clrs = ['blue','red','cyan','green','magenta', 'black']

    fig=plt.figure()
    ax=plt.subplot(111)
    span = 0.09
    for k in range(len(pair_interaction_times)):

        # multiple lines all full height
        temp = pair_interaction_times[k]
        chunk_start = []
        chunk_end = []
        chunk_start.append(temp[0])
        for p in range(1, temp.shape[0],1):
            if temp[p]-temp[p-1]>1:
                #print (temp[p], temp[p-1])
                chunk_end.append(temp[p-1])
                chunk_start.append(temp[p])
        chunk_end.append(temp[-1])

        chunk_start=np.array(chunk_start)/25.
        chunk_end=np.array(chunk_end)/25.

        for p in range(len(chunk_start)):
            ax.axvspan(chunk_start[p], chunk_end[p], ymin=k*span, ymax=k*span+span, alpha=0.5, color=clrs[0])

    # Interactions triples
    start1 = k*span+span
    for k in range(len(triples_interaction_time)):

        # multiple lines all full height
        temp = np.unique(triples_interaction_time[k])
        #print (temp)
        chunk_start = []
        chunk_end = []
        chunk_start.append(temp[0])
        for p in range(1, temp.shape[0],1):
            if temp[p]-temp[p-1]>1:
                #print (temp[p], temp[p-1])
                chunk_end.append(temp[p-1])
                chunk_start.append(temp[p])
        chunk_end.append(temp[-1])

        chunk_start=np.array(chunk_start)/25.
        chunk_end=np.array(chunk_end)/25.

    #     print ("# chunks: ", len(chunk_start), clrs[k])   
    #     print (chunk_start[:3], chunk_end[:3])

        for p in range(len(chunk_start)):
            ax.axvspan(chunk_start[p], chunk_end[p], ymin=start1+k*span, ymax=start1+k*span+span, alpha=0.5, color=clrs[1])


    # Interactions triples
    start2 = start1+k*span+span
    for k in range(len(quad_interaction_times)):

        # multiple lines all full height
        temp = np.unique(quad_interaction_times[k])
        chunk_start = []
        chunk_end = []
        chunk_start.append(temp[0])
        for p in range(1, temp.shape[0],1):
            if temp[p]-temp[p-1]>1:
                #print (temp[p], temp[p-1])
                chunk_end.append(temp[p-1])
                chunk_start.append(temp[p])
        chunk_end.append(temp[-1])

        chunk_start=np.array(chunk_start)/25.
        chunk_end=np.array(chunk_end)/25.

    #     print ("# chunks: ", len(chunk_start), clrs[k])   
    #     print (chunk_start[:3], chunk_end[:3])

        for p in range(len(chunk_start)):
            ax.axvspan(chunk_start[p], chunk_end[p], ymin=start2+k*span, ymax=start2+k*span+span, alpha=0.5, color=clrs[3])

    plt.xlim(0,3600.)
    plt.ylim(0,1)

    handles=[]
    patch = mpatches.Patch(color=clrs[0], label='pairs')
    handles.append(patch) 
    patch = mpatches.Patch(color=clrs[1], label='triples')
    handles.append(patch) 
    patch = mpatches.Patch(color=clrs[3], label='quads')
    handles.append(patch) 

    # plot the legend   
    plt.legend(handles=handles,fontsize=20)
    plt.tick_params(labelsize=20)
    plt.xlabel("Time (sec)",fontsize=20)
    plt.yticks([])

    plt.show()

In [3]:
# Classifier output for CSUA; using 100 x 100 pixel images;
root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed'

#            
locs = np.load(root_dir + '/locs_array_clean.npy')
print (locs.shape)

(4, 89987, 2)


In [4]:
############################################
############ VISUALIZE TRACES ##############
############################################
clrs = ['blue','red','yellow','green','cyan']
names= ['female','male','pup1','pup2','block']
window = 10 * 25
metre_per_pixel = 0.5*1E-3

for p in range(4):
    
    # compute differences as movement
    positions = locs[p].sum(1)
    t = np.arange(positions.shape[0])/25.
    print (p, positions)
    plt.plot(t, positions, linewidth=3, c=clrs[p], label=names[p])
plt.legend(fontsize=20)
plt.show()

0 [1355.4294 1355.4294 1355.4294 ... 1339.2007 1349.5914 1355.4294]
1 [748.4761 748.4761 748.4761 ... 748.4761 748.4761 748.4761]
2 [971.3129  971.3129  971.3129  ... 972.9657  973.03876 971.3129 ]
3 [889.2394  889.2394  889.2394  ... 889.7843  889.29144 889.2394 ]


In [5]:
############################################
############ MAKE VIDEO BASES ON TRACES ####
############################################
#          pup1     pup2    female  male
# go through first videos
from tqdm import trange

colors_4= ['orange','green', 'blue', 'red', 'cyan']
colors_4= ['blue','red', 'cyan', 'green', 'yellow']

video_name = root_dir + '/' + os.path.split(root_dir)[1]+'.avi'
print ("video name: ", video_name)
original_vid = cv2.VideoCapture(video_name)
font = cv2.FONT_HERSHEY_PLAIN

# video sizes
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale

#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'

start = 49375
end = start+1000
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

# Initialize video out
fname_out = video_name[:-4]+"_locs_added_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)


for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    for k in range(locs.shape[0]):
        y = locs[k,n,0]
        x = locs[k,n,1]
        
        if np.isnan(x) or np.isnan(y):
            continue
        else:
            x=int(x)//scale
            y=int(y)//scale
            
            frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(colors_4[k]))*255.).astype('uint8')
            #print (colors_4[k])
            #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')
                
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()



video name:  /media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi


100%|██████████| 1000/1000 [00:22<00:00, 45.01it/s]


In [11]:
############################################
####### MAKE BEHAVIOUR CYCLE PLOTS #########
############################################

clrs = ['blue','red','cyan','green','magenta']
names= ['female','male','pup1','pup2','block']
window = 60 * 25
metre_per_pixel = 0.5*1E-3

for p in range(locs.shape[0]):
    
    # compute differences as movement
    positions = locs[p]
    velocities = np.sqrt((positions[1:,0]-positions[:-1,0])**2+
                         (positions[1:,1]-positions[:-1,1])**2)
    print (velocities)
                         
    ynew = np.convolve(velocities, np.ones((window,))/window, mode='valid')
    t = np.arange(ynew.shape[0])/25.
        
    dists = np.sum(ynew)
    print (dists)
    
    plt.plot(t, ynew, linewidth=3, c=clrs[p], 
             label=names[p]+" dist: "+str(round(dists*metre_per_pixel,1)) + " m")

# label information
plt.ylabel("Activity", fontsize=30)
plt.yticks([0],fontsize=30)
plt.legend(fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)
plt.xlim(t[0],t[-1])
plt.ylim(bottom=0)
plt.suptitle("Movement analysis for "+os.path.split(root_dir)[1] + "\n" +
             str(int(window/25))+"sec smoothing window",fontsize=25)
plt.show()
    

[0.        0.        0.        ... 5.8767567 7.3662534 4.299903 ]
251109.22320292855
[ 0.        0.        0.       ... 16.684471  0.        0.      ]
328115.36823327956
[0.         0.         0.         ... 0.39229134 0.06914642 1.5767915 ]
253086.72509087785
[0.        0.        0.        ... 3.8562996 0.3494774 0.1629075]
269696.3865440486


In [9]:
###############################################
# COMPUTE SOCIAL INTERACTIONS BETWEEN ANIMALS###
################################################

(pair_interaction_times,triples_interaction_time, quad_interaction_times) = plot_pairwise_interactions(locs)

    

(89987, 2)
pair_:  (0, 1) (26205,)
pair_:  (0, 2) (44387,)
pair_:  (1, 2) (16245,)
(0, 1, 2) IDX3 4:  3659
pair_:  (0, 1) (26205,)
pair_:  (0, 2) (40615,)
pair_:  (1, 2) (27836,)
(0, 1, 3) IDX3 4:  11776
pair_:  (0, 1) (44387,)
pair_:  (0, 2) (40615,)
pair_:  (1, 2) (37784,)


<ipython-input-2-d0bec4a2ef7f>:98: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  idx3 = np.unique(np.hstack(set.intersection(*[set(x) for x in idx_array])))


(0, 2, 3) IDX3 4:  24156
pair_:  (0, 1) (16245,)
pair_:  (0, 2) (27836,)
pair_:  (1, 2) (37784,)
(1, 2, 3) IDX3 4:  4708

(0, 1, 2, 3)
pair2:  (0, 1) (26205,)
pair2:  (0, 2) (44387,)
pair2:  (0, 3) (40615,)
pair2:  (1, 2) (16245,)
pair2:  (1, 3) (27836,)
pair2:  (2, 3) (37784,)
idx_array:  [array([   43,    44,    45, ..., 89104, 89105, 89106]), array([   42,   427,   428, ..., 89845, 89846, 89847]), array([   42,    43,    44, ..., 89811, 89812, 89813]), array([    0,     1,     2, ..., 89984, 89985, 89986]), array([    0,     1,     2, ..., 89984, 89985, 89986]), array([    0,     1,     2, ..., 89984, 89985, 89986])]
4 animals 4:  948

4 ['female', 'male', 'pup1', 'pup2']


In [16]:
#######################################
####### TIME SPENT TOGETHER PLOTS #####
#######################################

ethogram_social_huddling(pair_interaction_times,
                         triples_interaction_time,
                         quad_interaction_times)
    

In [19]:
############################
####### SMOOTH TRACES ######
############################
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


box_width = 25
print (locs.shape)
locs_smooth = locs.copy()
for k in range(locs.shape[0]):
    for p in range(locs.shape[2]):
        locs_smooth[k,:,p] = smooth(locs_smooth[k,:,p],box_width)


(4, 89987, 2)


In [18]:
##############################################################
####### FIND MOVEMENT INITIATIONS AND WHO FOLLOWS WHO ########
##############################################################
        
# compute velocities for each animal
vel = []
for k in range(locs_smooth.shape[0]):
    temp = locs_smooth[k]
    temp_vel = np.sqrt((temp[1:,0]-temp[:-1,0])**2+
                       (temp[1:,1]-temp[:-1,1])**2)
    vel.append(temp_vel)
    
vel = np.array(vel)
# print (vel.shape)
# for k in range(4):
#     plt.plot(vel[k])
# plt.show()

clrs = ['red','blue','cyan','green','magenta', 'black']
animal_names = ['female','male','pup1','pup2']

# detect movement from any period of rest
frame_rate = 25
min_rest_period = 5 * frame_rate

# walk vs. running thresholds
rest_velocity = 3  #very slowly moving / stationary threshold
                   # measured in pixels/frame movement; 
min_rest_duration = int(.25 * frame_rate)  # movement < n frames between rest will be discarded (i.e. counted as rest)
run_velocity = 30

# detect movement initiations from rest
mov_initiations = np.zeros((4, locs.shape[1]),'float32')
rest_array = []
for k in range(4):
    temp = vel[k]
    idx = np.where(temp<=rest_velocity)[0]
    print (k, idx.shape)
    
    # compute time differences for rest periods; search for periods > some duration to detect  rather than
    # artifactual trace jitter
    diff = idx[1:]-idx[:-1]
    
    rest_start = []
    rest_end = []
    rest_start.append(idx[0])
    for p in range(1,idx.shape[0],1):
        # check to see if moving to new rest state
        if (idx[p]-idx[p-1])>min_rest_duration:
            rest_end.append(idx[p-1])
            rest_start.append(idx[p])
    rest_end.append(idx[p])
    
    #
    rests = np.zeros((len(rest_start),2),'int32')
    rests[:,0] = rest_start
    rests[:,1] = rest_end
    rests = rests
    print (k, rests.shape)
    

    rest_array.append(rests)
    # connect all rests periods that are less than threshold apart:
    mov_initiations[k,rests[:,1]] += 1
    #for p in range(rests.shape[0]):
    
    # plot movement intiations as the end of rest periods:
    #plt.scatter(rests[:,1],rests[:,1]*0+k, c=clrs[k])
    
print (mov_initiations.shape)
#plt.show()

# PROCESS MOVEMENTS
fig = plt.figure()
frame_rate = 25
n_sec_window = 1

# make matrix to hold 10 second window of time. 
matrix = np.zeros((4,4),'float32')  # no. of animals, x no. of 
# loop over all animals
for k in range(len(rest_array)):
    # loop over each rest period
    for p in range(rest_array[k].shape[0]):
        # loop over all other animals        
        for a in range(4):
            diffs = rest_array[k][p,1]-rest_array[a][:,1]
            idx = np.where(diffs>=0)[0]
            #
            if idx.shape[0]>0:
                diffs=diffs[idx]
                # check 
                if np.min(diffs)<=(frame_rate*n_sec_window):
                    matrix[k,a]+=1
            
#
for j in range(4):
    matrix[j,j]=np.nan
plt.imshow(matrix)
plt.colorbar()

x_ticks=['female','male','pup1','pup2']
plt.xticks(np.arange(4),x_ticks)
plt.yticks(np.arange(4),x_ticks)
plt.ylabel("Leader")
plt.xlabel("Follower")


fig = plt.figure()
for k in range(4):
    print (matrix[k]/rest_array[k].shape[0])
    plt.scatter(np.arange(4), 
                matrix[k]/rest_array[k].shape[0], 
                s=100,
                c=clrs[k], label=animal_names[k])

#x_ticks=['female','male','pup1','pup2']
plt.xticks(np.arange(4),x_ticks)

plt.legend(fontsize=20)
plt.ylim(bottom=0)
plt.title("Probability of following leader within "+ 
          str(n_sec_window) + 'sec' , fontsize=20)
plt.xlabel("Leader",fontsize=20)
plt.tick_params(labelsize=20)
plt.ylabel("Probability (%)", fontsize=20)
plt.show()

    

(4, 89987, 2)
0 (82159,)
0 (241, 2)
1 (79544,)
1 (338, 2)
2 (79705,)
2 (342, 2)
3 (77868,)
3 (368, 2)
(4, 89987)
[       nan 0.20746888 0.32365146 0.20746888]
[0.16863905        nan 0.23076923 0.22485207]
[0.19005848 0.21637426        nan 0.25146198]
[0.1521739  0.22826087 0.23097827        nan]


In [68]:
################################################################
####### FIND PROBABILITIES OF CHASING WITHIN 10 SEC OF MOVING ##
################################################################
# make matrix to hold 10 second window of time. 
matrix = np.zeros((4,11,4),'float32')  # no. of animals, x no. of animals, x 10 sec
# loop over all animals
for k in range(len(rest_array)):
    # loop over each rest period
    for p in range(rest_array[k].shape[0]):
        # loop over all other animals        
        for a in range(4):
            diffs = rest_array[k][p,1]-rest_array[a][:,1]
            idx = np.where(diffs>=0)[0]
            #
            if idx.shape[0]>0:
                diffs=diffs[idx]
                
                # check 
                #if np.min(diffs)<=(frame_rate*n_sec_window):
                # add all the values
                for q in range(0,11,1):
                    idx3 = np.where(np.logical_and(diffs>=(q*frame_rate),
                                                  diffs<((q+1)*frame_rate)))[0]
                    matrix[k,q,a]+=idx3.shape[0]
            
#
fig=plt.figure()
ax=plt.subplot()
fontsize=30
#for animal in range(4):
t=np.arange(11)+0.5
for animal in range(1):
    #ax=plt.subplot(4,1,animal+1)
    for k in range(4):
        if k==animal:
            continue
        plt.plot(t,matrix[animal,:,k]/rest_array[k].shape[0],
                 linewidth=5,
                label=animal_names[k] + " follows "+animal_names[animal])
    plt.legend(fontsize=fontsize)
    plt.xticks([])
    plt.xlim(0,10)
plt.xticks(np.arange(11))
plt.tick_params(labelsize=fontsize)
plt.ylim(bottom=0)
ax.axvspan(0,1, ymin=0, ymax=1, alpha=0.1, color='black')

plt.plot([0,10],[0.1, 0.1],'r--',linewidth=5, c='black')

plt.xlabel("Time (sec)", fontsize=fontsize)
plt.title("Predicting chase behaviour", fontsize=fontsize)
plt.show()

In [27]:
# detect animal interactions and who leaves firs

# detect when animals come together;

print (pair_interaction_times[0].shape)
#locs_smooth

for k in range(len(pair_interaction_times)):

    # multiple lines all full height
    temp = pair_interaction_times[k]
    chunk_start = []
    chunk_end = []
    chunk_start.append(temp[0])
    for p in range(1, temp.shape[0],1):
        if temp[p]-temp[p-1]>1:
            #print (temp[p], temp[p-1])
            chunk_end.append(temp[p-1])
            chunk_start.append(temp[p])
    chunk_end.append(temp[-1])

    chunk_start=np.array(chunk_start)/25.
    chunk_end=np.array(chunk_end)/25.

    inter = np.zeros((chunk_start.shape[0],2),'float32')
    inter[:,0]=chunk_start
    inter[:,1]=chunk_end
    print (inter.shape)
    
    


(26205,)
(179, 2)
(143, 2)
(135, 2)
(144, 2)
(149, 2)
(170, 2)


In [ ]:
# detect animal interactions and who leaves firs


In [11]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy

animal_names = ['female','male','pup1','pup2']

#df = pd.read_csv('synchrony_sample.csv')
for k1 in range(4):
    for k2 in range(k1+1,4,1):
        r, p = stats.pearsonr(vel[k1], vel[k2])
        #r, p = scipy.signal.correlate2d(locs[k1],locs[k2])
        print(animal_names[k1] + " " + animal_names[k2], r, p )


female male 0.1666545428180521 0.0
female pup1 0.17186924396736675 0.0
female pup2 0.1820420485776153 0.0
male pup1 0.247553286152024 0.0
male pup2 0.1897036038109263 0.0
pup1 pup2 0.1584994640732148 0.0


In [85]:
print (vel.shape)

(4, 89986)


In [12]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

import pandas as pd
        
# create an array 
print (vel.shape)
print (mov_initiations.shape)
f,ax=plt.subplots(figsize=(14,3))
for k1 in range(4):
    for k2 in range(k1+1, 4, 1):
        dataset = pd.DataFrame()
        dataset['col1'] = mov_initiations[k1]
        dataset['col2'] = mov_initiations[k2]
        dataset.head()

        d1 = dataset['col1']
        d2 = dataset['col2']

        seconds = 1
        fps = 25
        rs = [crosscorr(d1,d2, lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
        offset = np.ceil(len(rs)/2)-np.argmax(rs)

        # plotting
        t= np.arange(-int(seconds*fps),int(seconds*fps+1))
        ax.plot(t,rs)
        ax.axvline(np.ceil(len(rs)/2),color='k',linestyle='--',label='Center')
        ax.axvline(t[np.argmax(rs)],color='r',linestyle='--',label='Peak synchrony')

ax.set(title=f'Offset = {offset} frames\nS1 leads <> S2 leads',
       ylim=[0.0,.2],
       #xlim=[0,seconds*fps*2], 
       xlabel='Offset',
       ylabel='Pearson r')
    #ax.set_xticks([0, 50, 100, 151, 201, 251, 301])
#ax.set_xticklabels([-150, -100, -50, 0, 50, 100, 150]);
plt.legend()
plt.plot()

(4, 89986)
(4, 89987)


[]